In [ ]:
import panel as pn
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go
from bokeh.io import output_notebook

output_notebook()
pn.extension("plotly")

In [ ]:
sp500 = (
    pd.read_csv(
        "3monthanalysis.csv", parse_dates=["Date"], dtype={"ticker": "category"}
    )
    .set_index(["ticker"])
    .sort_index()
)

In [ ]:
def create_plot(ticker="NVDA", threshold=1):
    threshold = (threshold + 0.01) / 100
    data = sp500.loc[ticker].set_index("Date")

    # add extra dates to avoid keyerrors
    # not working. shifts the df and removes the first ten days
    # data.index = data.index + pd.Timedelta(days=10)
    start, stop = data.index.min(), (data.index.max() + pd.offsets.BusinessDay(1))
    date_range = pd.date_range(start, stop)
    data = data.reindex(date_range)
    # get the days that went unfilled and filter threshold paramter
    unfilled = data[data.fill_date.isna()].query("abs_change >= @threshold")

    # create three lists to create the gap vrect and line
    xs = []
    ys = []
    vrect_xs = []
    for date, e in unfilled[["Close", "Open"]].iterrows():
        tom = date + pd.Timedelta(days=1)
        xs.extend([date, tom, None])
        ys.extend([e.Close, data.loc[tom].Open, None])
        vrect_xs.append((date, tom))

    # plotly playout
    layout = go.Layout(
        title="",
        autosize=False,
        width=1200,
        height=600,
        margin=dict(t=50, b=50, r=50, l=50),
        xaxis_rangeslider_visible=False,
    )

    candlestick = go.Candlestick(
        x=data.index,
        open=data.Open,
        high=data.High,
        low=data.Low,
        close=data.Close,
        # opacity=1,
    )

    gap_lines = go.Scatter(
        x=xs,
        y=ys,
        mode="lines",
        line={"width": 3},
        marker_color="rgb(0,0,0)",
    )

    # this is how plotly wants a figure
    fig = go.Figure(data=[candlestick, gap_lines], layout=layout)

    # add the vrects to the figure
    for (
        today,
        tom,
    ) in vrect_xs:
        fig.add_vrect(
            x0=today,
            x1=tom,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),

    # create the panel and return it
    plotly_pane = pn.pane.Plotly(fig)
    return plotly_pane


# ticker choice widget
tickers = sp500.index.get_level_values("ticker").unique().to_list()
ticker_widget = pn.widgets.Select(name="ticker", value="NVDA", options=tickers)

# threshold widget
threshold_widget = pn.widgets.IntSlider(name="threshold", value=3, start=0, end=10)

bound_plot = pn.bind(create_plot, ticker=ticker_widget, threshold=threshold_widget)
app = pn.Column(ticker_widget, threshold_widget, bound_plot).servable()

app